In [2]:
%pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [1]:
# General png to jpg size down scale

import os
from PIL import Image

INPUT_DIR = "/Users/haithammohamedibrahim/VsCode/poems/Faz3-Poems/images/Poems/Focus"
OUTPUT_DIR = "/Users/haithammohamedibrahim/VsCode/poems/Faz3-Poems/images/Poems/Focus/down"

os.makedirs(OUTPUT_DIR, exist_ok=True)

for filename in os.listdir(INPUT_DIR):
    if filename.lower().endswith((".jpg", ".jpeg", ".png", ".tif", ".tiff", ".bmp", ".webp", ".webm", ".webp")):
        input_path = os.path.join(INPUT_DIR, filename)
        output_name = os.path.splitext(filename)[0] + ".jpg"
        output_path = os.path.join(OUTPUT_DIR, output_name)

        img = Image.open(input_path)

        if img.mode != "RGB":
            img = img.convert("RGB")

        img.save(
            output_path,
            format="JPEG",
            quality=50,          # Photoshop High (8)
            subsampling=2,       # 4:2:0
            optimize=True,
            progressive=True
        )

print("Done.")

Done.


In [4]:
# thumbnails only down scale
from PIL import Image
import os

INPUT_DIR = "/Users/haithammohamedibrahim/VsCode/poems/Faz3-Poems/images/Poems/thumbnail"
OUTPUT_DIR = f"{INPUT_DIR}/downscaled"
DOWNSAMPLE_FACTOR = 8

os.makedirs(OUTPUT_DIR, exist_ok=True)

for filename in os.listdir(INPUT_DIR):
    if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff", ".webp")):
        input_path = os.path.join(INPUT_DIR, filename)
        output_path = os.path.join(OUTPUT_DIR, filename)

        with Image.open(input_path) as img:
            w, h = img.size
            new_size = (w // DOWNSAMPLE_FACTOR, h // DOWNSAMPLE_FACTOR)

            img_resized = img.resize(new_size, Image.LANCZOS)
            img_resized.save(output_path)

print("Downsampling complete.")

Downsampling complete.


In [ ]:
# ai rename ar to en_ar

import os
import requests

# LM Studio local API
LM_STUDIO_URL = "http://localhost:1234/v1/chat/completions"
MODEL_NAME = "qwen/qwen3-4b-2507"  # replace with your loaded model

def translate_to_english(arabic_text):
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {
                "role": "system",
                "content": (
                    "You are a professional english translator with focus in UAE attire Poetry. "
                    "Take the following Arabic text (from a filename) and return ONLY its natural English translation. "
                    "Do not add any commentary or extra characters. Never add '_' between english translation words, only sperate english words with space "
                )
            },
            {
                "role": "user",
                "content": arabic_text
            }
        ],
        "temperature": 0.1
    }

    r = requests.post(LM_STUDIO_URL, json=payload, timeout=60)
    r.raise_for_status()
    return r.json()["choices"][0]["message"]["content"].strip()

# Folder containing Arabic-named images
folder = "/Users/haithammohamedibrahim/VsCode/poems/Faz3-Poems/images/Poems/thumbnail"

for filename in os.listdir(folder):
    if "_" in filename:
        continue  # skip already-processed files

    name, ext = os.path.splitext(filename)
    name = name.strip()

    try:
        english = translate_to_english(name)
        # Clean English text to be safe for filenames
        english_safe = "".join(c for c in english if c.isalnum() or c in " _-").strip()
    except Exception as e:
        print(f"❌ Failed to translate {filename}: {e}")
        continue

    new_name = f"{english_safe}_{name}{ext}"
    os.rename(
        os.path.join(folder, filename),
        os.path.join(folder, new_name)
    )
    print(f"✔ {filename} → {new_name}")

✔ بياض التجربه.png → White Experience_بياض التجربه.png
✔ ابن فراج.png → Son of Faraj_ابن فراج.png
✔ صبري ملني.png → Sabi meleni_صبري ملني.png
✔ من امس.png → From yesterday_من امس.png
✔ اقرب تعال.png → Come closer_اقرب تعال.png
✔ هذا الشبل من ذاك الاسد.png → This Shawl from that Lion_هذا الشبل من ذاك الاسد.png
✔ الوطن كله ابوك.png → The whole nation is your father_الوطن كله ابوك.png
✔ الجروح.png → Wounds_الجروح.png
✔ الاسم.png → Name_الاسم.png
✔ الله لنا.png → God with us_الله لنا.png
✔ شهر سته.png → Month six_شهر سته.png
✔ الصوره.png → The picture_الصوره.png
✔ رد حمدان بن محمد علي ابياتضاحي خلفان.png → Response by Hamdan Bin Mohammed Ali Abi Taha Khalwan_رد حمدان بن محمد علي ابياتضاحي خلفان.png
✔ فنون الغدر.png → Art of Deception_فنون الغدر.png
✔ رساله حزن.png → Message of sorrow_رساله حزن.png
✔ رمز العز علمني.png → Symbol of Pride taught me_رمز العز علمني.png
✔ كويت العز.png → Kuwait the Pride_كويت العز.png
✔ احمر الورد.png → Red Rose_احمر الورد.png
✔ عذروبها.png → Her sorrow_عذروبها.

In [ ]:
import os

GUIDE_DIR = "/Users/haithammohamedibrahim/VsCode/poems/Faz3-Poems/images/Poems/Focus"
TARGET_DIR = "/Users/haithammohamedibrahim/VsCode/poems/Faz3-Poems/images/Poems/Focus/down"

IMAGE_EXTS = {".png", ".jpg", ".jpeg", ".webp"}

def collect_basenames(folder):
    names = set()
    for fname in os.listdir(folder):
        base, ext = os.path.splitext(fname)
        if ext.lower() in IMAGE_EXTS:
            names.add(base.strip())
    return names

# Ground truth
guide_names = collect_basenames(GUIDE_DIR)

# What exists
target_names = collect_basenames(TARGET_DIR)

# Compare
missing_in_target = sorted(guide_names - target_names)
extra_in_target   = sorted(target_names - guide_names)

# Output
print("\n=== Missing in TARGET (expected from GUIDE) ===")
for name in missing_in_target:
    print(name)

print("\n=== Extra in TARGET (not in GUIDE) ===")
for name in extra_in_target:
    print(name)

print("\nSummary:")
print(f"Guide files : {len(guide_names)}")
print(f"Target files: {len(target_names)}")
print(f"Missing     : {len(missing_in_target)}")
print(f"Extra       : {len(extra_in_target)}")


=== Missing in TARGET (expected from GUIDE) ===
Peak of Union 2_قمه الالتحام(2)
atomic bomb_قنبله ذريه.jpg

=== Extra in TARGET (not in GUIDE) ===
Faman allah_فمان الله
atomic bomb_قنبله ذريه

Summary:
Guide files : 353
Target files: 353
Missing     : 2
Extra       : 2


In [ ]:
# checking if thumbnail names match Focus

import os

GUIDE_DIR = "/Users/haithammohamedibrahim/VsCode/poems/Faz3-Poems/images/Poems/thumbnail"
TARGET_DIR = "/Users/haithammohamedibrahim/VsCode/poems/Faz3-Poems/images/Poems/Focus"

IMAGE_EXTS = {".png", ".jpg", ".jpeg", ".webp"}

def collect_names(folder):
    names = set()
    for fname in os.listdir(folder):
        base, ext = os.path.splitext(fname)
        if ext.lower() in IMAGE_EXTS:
            names.add(base.strip())
    return names

# Guide is ground truth
guide_names = collect_names(GUIDE_DIR)
target_names = collect_names(TARGET_DIR)

# Compare
missing_in_target = sorted(guide_names - target_names)
extra_in_target   = sorted(target_names - guide_names)

# Output
print("\n=== ❌ Missing in TARGET (should exist) ===")
for name in missing_in_target:
    print(name)

print("\n=== ⚠️ Extra in TARGET (unexpected) ===")
for name in extra_in_target:
    print(name)

print("\n=== ✅ SUMMARY ===")
print(f"Guide count  : {len(guide_names)}")
print(f"Target count : {len(target_names)}")
print(f"Missing      : {len(missing_in_target)}")
print(f"Extra        : {len(extra_in_target)}")
print(f"Perfect match: {len(guide_names & target_names)}")


=== ❌ Missing in TARGET (should exist) ===

=== ⚠️ Extra in TARGET (unexpected) ===

=== ✅ SUMMARY ===
Guide count  : 352
Target count : 352
Missing      : 0
Extra        : 0
Perfect match: 352


In [2]:
import pandas as pd
import random

def create_poems_csv_with_similar():
    poem_ids = list(range(1, 354))
    related_poems_list = []
    
    for poem_id in poem_ids:
        candidates = [i for i in range(1, 354) if i != poem_id]
        num_related = random.randint(5, 20)
        related_ids = random.sample(candidates, num_related)
        
        related_list = [f"{pid}:{rank+1}" for rank, pid in enumerate(related_ids)]
        related_str = "[" + ", ".join(related_list) + "]"
        related_poems_list.append(related_str)
    
    df = pd.DataFrame({
        'poem_id': poem_ids,
        'related_poems': related_poems_list
    })
    
    df.to_csv('poems_with_similar.csv', index=False)
    return df

df = create_poems_csv_with_similar()